<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

想把 HuggingFaceTB/cosmopedia 英文数据中的prompt和text 翻译成 中文， 然后看了下 deep_translator的实现， 翻译调用的是三方接口集成库， 于是使用这个库封装的谷歌翻译接口来翻译，但是三方平台接口多会有限流和接口调用频率限制，即使在代码中有容错处理， 比如常规的sleep下再调用，不影响整理处理流程，但是整体处理时间受接口限制，即使用批处理也如此，这个在大规模数据处理中使用三方接口时，经常会遇到的问题，用的三方服务，在技术上不好解决； 于是选择另外一种方案，看是否有开源的翻译模型，底层模型结构一般也是Transform架构 Encoder-decoder model ，也称sequence-to-sequence model； 比如 谷歌的T5模型， 但是推理速度受硬件条件影响，比较慢， 然后看了下meta nllb 模型，专门用来处理翻译的模型，单个 AI 模型中支持 200 种语言，开源地址： https://github.com/facebookresearch/fairseq/tree/nllb  

模型相对现在的LLM参数量小一些，也在huggingface的Transforms库中集成 nllb-200-distilled-600M，直接可以load使用， 等等。。。 既然llm推理可以通过想llama.cpp通过加载ggml格式进行量化，在性能有少许折损的情况下降低推理成本，但是ggml格式还不支持nllb模型权重文件；那就直接用Transforms库来加载facebook/nllb-200-distilled-600M 模型来批量翻译试试看

- 使用Transforms库进行翻译任务： https://huggingface.co/docs/transformers/tasks/translation

- https://github.com/facebookresearch/fairseq/tree/nllb

- https://ai.meta.com/blog/nllb-200-high-quality-machine-translation/

后续尝试使用 [' Helsinki-NLP/opus-mt-en-zh '](https://huggingface.co/Helsinki-NLP/opus-mt-en-zh) 模型，


In [1]:
!pip install -q -U datasets deep-translator transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00


In [2]:
!huggingface-cli download \
  --repo-type dataset HuggingFaceTB/cosmopedia data/stories/train-00000-of-00043.parquet \
  --local-dir ./datas/datasets/HuggingFaceTB/cosmopedia \
  --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/datasets/HuggingFaceTB/cosmopedia/resolve/main/data/stories/train-00000-of-00043.parquet to /root/.cache/huggingface/hub/tmp_65cfxuf
train-00000-of-00043.parquet: 100% 277M/277M [00:01<00:00, 190MB/s]
./datas/datasets/HuggingFaceTB/cosmopedia/data/stories/train-00000-of-00043.parquet


In [ ]:
!cat translate.py

# https://huggingface.co/learn/nlp-course/chapter5/
# https://zhuanlan.zhihu.com/p/564816807

from deep_translator import GoogleTranslator
from datasets import load_dataset, load_from_disk
import argparse
import time


def translate_en2cn(item):
    translated = ""
    try_cn = 100
    while try_cn > 0:
        try:
            translated_prompt = GoogleTranslator(source='en', target='zh-CN').translate(
                item["prompt"])
            translated_text = GoogleTranslator(source='en', target='zh-CN').translate(
                item["text"])
            break
        except Exception as e:
            print("An error occurred:", e)
            time.sleep(1)
            try_cn -= 1

    return {"text_zh": translated_text, "prompt_zh": translated_prompt}


def batch_check_data(batch):
    return [len(item) != 0 for item in batch["text_zh"]]


def translate2save(src_dataset_dir: str, target_dataset_dir: str):
    data = load_dataset(src_dataset_dir, split="train[:100]")
    data =

In [ ]:
!python translate.py translate -s ./datas/datasets/HuggingFaceTB/cosmopedia/data/stories \
  -t ./datas/datasets/HuggingFaceTB/cosmopedia_zh

Map: 100% 100/100 [03:34<00:00,  2.15s/ examples]
Saving the dataset (1/1 shards): 100% 100/100 [00:00<00:00, 9226.77 examples/s]
Dataset({
    features: ['text', 'prompt', 'text_token_length', 'seed_data', 'format', 'audience', 'text_zh', 'prompt_zh'],
    num_rows: 100
})


## facebook/nllb

In [22]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", use_auth_token=True, src_lang="eng_Latn",target_lang="zho_Hans")
tokenizer


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:732: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


NllbTokenizerFast(name_or_path='facebook/nllb-200-distilled-600M', vocab_size=256204, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['ace_Arab', 'ace_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'afr_Latn', 'ajp_Arab', 'aka_Latn', 'amh_Ethi', 'apc_Arab', 'arb_Arab', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'asm_Beng', 'ast_Latn', 'awa_Deva', 'ayr_Latn', 'azb_Arab', 'azj_Latn', 'bak_Cyrl', 'bam_Latn', 'ban_Latn', 'bel_Cyrl', 'bem_Latn', 'ben_Beng', 'bho_Deva', 'bjn_Arab', 'bjn_Latn', 'bod_Tibt', 'bos_Latn', 'bug_Latn', 'bul_Cyrl', 'cat_Latn', 'ceb_Latn', 'ces_Latn', 'cjk_Latn', 'ckb_Arab', 'crh_Latn', 'cym_Latn', 'dan_Latn', 'deu_Latn', 'dik_Latn', 'dyu_Latn', 'dzo_Tibt', 'ell_Grek', 'eng_Latn', 'epo_Latn', 'est_Latn', 'eus_Latn', 'ewe_Latn', 'fao_Latn', 'pes_Arab'

In [23]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M", use_auth_token=True)
model

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): Embedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): Embedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm

In [30]:
%%time
article = [
    """
    Dr. Mitchell nodded approvingly. "Yes, indeed! Fatty fish boast rich stores of omega-3 fatty acids – specifically EPA and DHA – which work miracles within our bodies.
    Amongst numerous benefits, they decrease triglyceride levels, lower blood pressure, discourage plaque formation, and even suppress irregular heartbeats."
    """,
    """
    An intriguing idea took root in Alex's mind, blossoming rapidly into resolve. If these simple changes could make such profound differences in his life,
    why stop there? Why not share this gift with others – friends, colleagues, strangers who might also benefit from understanding how food choices impact heart health?
    Perhaps he could host educational events, cooking demonstrations, or write articles championing these powerful culinary tools.
    Yes, he decided, that would be his mission – to spread awareness and inspire change, one plate at a time.
    """,
    """
    His thoughts must have shown on his face because Dr. Mitchell suddenly grinned, leaning back in her chair with satisfaction.
    "Ah, I see the wheels turning in that brilliant mind of yours, Alex. Go ahead, run with it!"
    """,
]
for item in article:
  print(len(item))

inputs = tokenizer(article, return_tensors="pt",padding=True,truncation=True)

translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["zho_Hans"], max_new_tokens=1024)
tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

335
574
232
CPU times: user 48.6 s, sys: 74.7 ms, total: 48.7 s
Wall time: 12.3 s


['密切尔博士 赞成. "是的,确实!脂肪鱼拥有丰富的欧米茄-3脂肪酸,特别是EPA和DHA,这些脂肪酸在我们的身体中能做出奇迹.',
 '亚历克斯的想法很有趣,迅速发展成决心.如果这些简单的变化可以在他的生活中产生如此深刻的差异,为什么不停下来?为什么不与他人分享这个礼物?朋友,同事,陌生人也可能会从了解食物选择如何影响心脏健康中受益?也许他可以举办教育活动,做饭示范,或写文章宣扬这些强大的食工具.',
 '他会在脸上看到他的想法,因为米切尔博士突然笑了,在椅子上仰卧,满意. ",我看到你的聪明智力转动,亚历克斯.']

## Helsinki-NLP/opus-mt

In [33]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_checkpoint = "Helsinki-NLP/opus-mt-en-zh"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-zh', vocab_size=65001, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	65000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [34]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [36]:
%%time
#text = ["In terms of time, the Chinese space station was built more than 20 years later than the International Space Station.","hello world."]
article = [
    """
    Dr. Mitchell nodded approvingly. "Yes, indeed! Fatty fish boast rich stores of omega-3 fatty acids – specifically EPA and DHA – which work miracles within our bodies.
    Amongst numerous benefits, they decrease triglyceride levels, lower blood pressure, discourage plaque formation, and even suppress irregular heartbeats."
    """,
    """
    An intriguing idea took root in Alex's mind, blossoming rapidly into resolve. If these simple changes could make such profound differences in his life,
    why stop there? Why not share this gift with others – friends, colleagues, strangers who might also benefit from understanding how food choices impact heart health?
    Perhaps he could host educational events, cooking demonstrations, or write articles championing these powerful culinary tools.
    Yes, he decided, that would be his mission – to spread awareness and inspire change, one plate at a time.
    """,
    """
    His thoughts must have shown on his face because Dr. Mitchell suddenly grinned, leaning back in her chair with satisfaction.
    "Ah, I see the wheels turning in that brilliant mind of yours, Alex. Go ahead, run with it!"
    """,
]
for item in article:
  print(len(item))

# Tokenize the text
inputs = tokenizer(article, return_tensors="pt",padding=True,truncation=True).input_ids

# Perform the translation and decode the output
translation = model.generate(inputs, max_new_tokens=512, do_sample=True, top_k=30, top_p=0.95)

result = tokenizer.batch_decode(translation, skip_special_tokens=True)
print(result)

335
574
232
['米切尔博士点头同意。 “是的,事实上,胖鱼有丰富的欧美加-3脂肪酸库 — — 特别是美国环保局和丹麦卫生局 — — 为我们的身体带来了奇迹。 在许多好处中,它们降低了三重血压水平,降低了血压,抑制了形成斑块,甚至抑制了不规则的心跳。 ”', '一种有趣的想法在亚历克斯的脑海中扎根,迅速发展成决心。 如果这些简单的变化能让他的生活产生如此深刻的差别,为什么停下来呢? 为什么不和其他人分享这一礼物 — — 朋友、同事、陌生人,他们可能也从了解食物选择如何影响心脏健康中受益? 也许他可以主办教育活动、烹饪示威或撰写文章支持这些强大的烹饪工具。 是的,他决定,这将是他的使命 — — 传播意识和激励变革,一个一个一盘一盘。', '他一定在脸上表现出了自己的想法,因为米切尔博士突然微笑, 满心满意地坐在她的椅子上。 “啊,我看到你的聪明头脑,亚历克斯, 转动了轮子。继续,去跑吧!”']
CPU times: user 20.6 s, sys: 48.8 ms, total: 20.7 s
Wall time: 5.18 s


## 总结：

facebook/nllb-200-distilled-600M 模型 翻译 效果一般，Helsinki-NLP/opus-mt-en-zh 整体效果还可以；以上翻译文本受文本长度限制，中文token可能由两个字组成(比如:"一种")，需要对长文本进行切割，然后在批量翻译，然后在组合；
在本地部署模型翻译的方式pass,需要加载参数大模型才能有好的翻译效果(和硬件无关，和模型推理有关)， 而且推理速度需要硬件GPU来加持，会有模型调参实现成本和计算成本在；
如果有机器支持，可以选用本地批量翻译的方式
如果使用三方翻译接口，批量处理就会受制于接口的限制（限速，翻译文本长度，并发请求数等等）
